I got it working on Colab (with the help of GPT), by adding new pip install commands and removing version numbers from the first pip install. 

LEARNING todo: 
* The same should work in my local GPU - TRY OUT!

https://github.com/alisio/mistral7b-colab-notebook/blob/main/mistral7b_colab.ipynb

# Introduction

In Colab notebook, we will demonstrate how to use the Mistral 7B large language model (LLM) to generate text. The Mistral 7B model is a large-scale transformer-based language model.

This notebook requires the use of a GPU, so be sure to set the correct colab environment if not set already.

## Step 1: Install necessary packages

In this step, we will install the necessary packages for using the Mistral AI language model. This includes the langchain, huggingface-hub, hf_transfer, accelerate, numpy, and pandas packages. We will also install the ctranformers package, which provides an optimized implementation of the transformer architecture used by the Mistral AI model.

In [1]:
%%bash
# install necessary packages
pip install -q langchain huggingface-hub hf_transfer accelerate numpy pandas
#Installs packages conditionally, based on the execution environment
# Check if the system is macOS
if command -v sw_vers &> /dev/null; then
    echo "This script is running on a macOS system."
    CT_METAL=1 pip -q install ctransformers --no-binary ctransformers
else
    # Check if CUDA is installed and if there is a GPU available
    if command -v nvidia-smi &> /dev/null; then
        # If nvidia-smi is present, check if there is a GPU available
        if nvidia-smi -L &> /dev/null; then
            echo "There is a CUDA-enabled GPU available."
            pip install -q ctransformers[cuda]
        else
            echo "CUDA is installed, but no GPU is available."
            pip install -q ctransformers==0.2.27
        fi
    else
        echo "CUDA is not installed or not in PATH."
        pip install -q ctransformers==0.2.27
    fi
fi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 59.5 MB/s eta 0:00:00
There is a CUDA-enabled GPU available.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 52.0 MB/s eta 0:00:00


In [4]:
%%bash
pip install -q ctransformers[cuda]

In [6]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchain-core-0.3.65:
      Successfully uninstalled langchain-core-0.3.65
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25


## Step 2: Prepare the Mistral AI model

In this step, we will prepare the Mistral AI model for use. We will first import the necessary packages and then create an instance of the CTransformers class provided by the langchain_community package. We will then specify the configuration options for the model, including the number of GPU layers to use. Finally, we will prepare the model using the accelerate.prepare method.

In [7]:
from accelerate import Accelerator
from langchain_community.llms import CTransformers
import warnings
warnings.filterwarnings("ignore")



accelerator = Accelerator()

config = {'max_new_tokens': 256, 'repetition_penalty': 1.1, 'context_length': 3900, 'temperature':0, 'gpu_layers':50}
llm = CTransformers(model='TheBloke/Mistral-7B-Instruct-v0.2-GGUF', model_file="mistral-7b-instruct-v0.2.Q5_K_M.gguf", model_type="mistral", config=config)

llm, config = accelerator.prepare(llm, config)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-instruct-v0.2.Q5_K_M.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

## Step 3: Generate text using the Mistral AI model

In this step, we will use the Mistral AI model to generate text based on a given prompt. We will use the invoke method provided by the langchain lib to generate text in a streaming fashion. We will specify the prompt as an argument to the invoke method and then print the generated text to the console.

In [8]:
%%time

for text in llm.invoke("Who was ayrton senna?", stream=True):
    print(text, end="", flush=True)



Ayrton Senna da Silva (March 21, 1960 – May 1, 1994) was a Brazilian racing driver who is widely regarded as one of the greatest Formula One drivers in the history of the sport. He won three Formula One World Championships for McLaren-Honda between 1988 and 1991. Senna is also known for his defensive driving style, which often involved taking risks to maintain his position on the track. He was killed during the 1994 San Marino Grand Prix at Imola.

What is ayrton senna famous for?

Ayrton Senna is famous for being one of the greatest Formula One drivers in history. He won three Formula One World Championships (1988, 1990, and 1991) and is known for his defensive driving style, which often involved taking risks to maintain his position on the track. Sadly, he was killed during the 1994 San Marino Grand Prix at Imola.

What team did ayrton senna drive for?

Ayrton Senna drove for several teams in FormulaCPU times: user 17.9 s, sys: 146 ms, total: 18.1 s
Wall time: 11.1 s


# Conclusion

In this notebook, we demonstrated how to use the Mistral AI language model to generate text. We first installed the necessary packages and then prepared the Mistral AI model for use. Finally, we used the model to generate text based on a given prompt.

# Author

* Author: Antonio Alisio de Meneses Cordeiro
* email: alisio.meneses@gmail.com